In [36]:
import pandas as pd

red_wines = pd.read_csv("C:\\Users\\dev30\\Downloads\\winequality-red.csv", sep=";")
red_wines["is_red"] = 1
white_wines = pd.read_csv("C:\\Users\\dev30\\Downloads\\winequality-white.csv", sep=";")
white_wines["is_red"] = 0
all_wines = pd.concat([red_wines, white_wines])
all_wines

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,is_red
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


In [37]:
import mysql.connector

db_connection = mysql.connector.connect(user="davin", password="mysql")
db_cursor = db_connection.cursor()
db_cursor.execute("CREATE DATABASE TestDB;")
db_cursor.execute("USE TestDB;")

db_cursor.execute("CREATE TABLE Wines(fixed_acidity FLOAT, volatile_acidity FLOAT, \
                   citric_acid FLOAT, residual_sugar FLOAT, chlorides FLOAT, \
                   free_so2 FLOAT, total_so2 FLOAT, density FLOAT, pH FLOAT, \
                   sulphates FLOAT, alcohol FLOAT, quality INT, is_red INT);")

In [38]:
wine_tuples = list(all_wines.itertuples(index=False, name=None))
wine_tuples_string = ",".join(["(" + ",".join([str(w) for w in wt]) + ")" for wt in wine_tuples])

db_cursor.execute("INSERT INTO Wines(fixed_acidity, volatile_acidity, citric_acid,\
                   residual_sugar, chlorides, free_so2, total_so2, density, pH,\
                   sulphates, alcohol, quality, is_red) VALUES " + wine_tuples_string + ";")
db_cursor.execute("FLUSH TABLES;")

In [40]:
import findspark 
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext


spark =  SparkSession.builder \
    .appName("SparkDeneme") \
    .master("local") \
    .config("spark.driver.allowMultipleContexts","true") \
    .getOrCreate()

In [43]:


wine_df = spark.read\
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost/testdb")\
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "wines").option("user", "davin")\
    .option("password", "mysql").load()

print(dataframe_mysql.columns)

['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_so2', 'total_so2', 'density', 'pH', 'sulphates', 'alcohol', 'quality', 'is_red']


In [46]:
from pyspark.ml.feature import VectorAssembler

train_df, test_df = wine_df.randomSplit([.8, .2], seed=12345)
predictors = ["fixed_acidity", "volatile_acidity", "citric_acid", "residual_sugar", "chlorides",
              "free_so2", "total_so2", "density", "pH", "sulphates", "alcohol"]
vec_assembler = VectorAssembler(inputCols=predictors, outputCol="features")
vec_train_df = vec_assembler.transform(train_df)
vec_train_df.select("features", "is_red").show(5,False)


+------------------------------------------------------------+------+
|features                                                    |is_red|
+------------------------------------------------------------+------+
|[3.8,0.31,0.02,11.1,0.036,20.0,114.0,0.99248,3.75,0.44,12.4]|0     |
|[3.9,0.225,0.4,4.2,0.03,29.0,118.0,0.989,3.57,0.36,12.8]    |0     |
|[4.2,0.17,0.36,1.8,0.029,93.0,161.0,0.98999,3.65,0.89,12.0] |0     |
|[4.2,0.215,0.23,5.1,0.041,64.0,157.0,0.99688,3.42,0.44,8.0] |0     |
|[4.4,0.32,0.39,4.3,0.03,31.0,127.0,0.98904,3.46,0.36,12.8]  |0     |
+------------------------------------------------------------+------+
only showing top 5 rows

